In [1]:
import pandas as pd
import numpy as np
import MySQLdb
from datetime import datetime
import os

In [47]:
db = MySQLdb.connect("localhost","root","123456")
db.set_character_set('gbk')

In [40]:
db = MySQLdb.connect("47.100.14.137","hyps44413","Nanhua")
db.set_character_set('gbk')

In [28]:
db.close()

In [31]:
dbname = ' m_oi_call_option'
cursor = db.cursor()

table_name = "`东方财富`"

sql = "ALTER TABLE " + dbname + ".v_rank ADD COLUMN " + table_name + " TEXT"
cursor.execute(sql)
sql = "ALTER TABLE " + dbname + ".b_rank ADD COLUMN " + table_name + " TEXT"
cursor.execute(sql)
sql = "ALTER TABLE " + dbname + ".s_rank ADD COLUMN " + table_name + " TEXT"
cursor.execute(sql)
sql = "ALTER TABLE " + dbname + ".v_increment ADD COLUMN " + table_name + " TEXT"
cursor.execute(sql)
sql = "ALTER TABLE " + dbname + ".b_increment ADD COLUMN " + table_name + " TEXT"
cursor.execute(sql)
sql = "ALTER TABLE " + dbname + ".s_increment ADD COLUMN " + table_name + " TEXT"
cursor.execute(sql)
sql = "ALTER TABLE " + dbname + ".v_volume ADD COLUMN " + table_name + " TEXT"
cursor.execute(sql)
sql = "ALTER TABLE " + dbname + ".b_oi ADD COLUMN " + table_name + " TEXT"
cursor.execute(sql)
sql = "ALTER TABLE " + dbname + ".s_oi ADD COLUMN " + table_name + " TEXT"
cursor.execute(sql)

999

In [48]:
dbname = 'm_oi_call_option'
cursor = db.cursor()
date = "'2018-05-03'"

sql = "delete from " + dbname + ".total_volume where Date >= " + date
cursor.execute(sql)

sql = "delete from " + dbname + ".v_rank where Date >= " + date
cursor.execute(sql)
sql = "delete from " + dbname + ".b_rank where Date >= " + date
cursor.execute(sql)
sql = "delete from " + dbname + ".s_rank where Date >= " + date
cursor.execute(sql)
sql = "delete from " + dbname + ".v_volume where Date >= " + date
cursor.execute(sql)
sql = "delete from " + dbname + ".b_oi where Date >= " + date
cursor.execute(sql)
sql = "delete from " + dbname + ".s_oi where Date >= " + date
cursor.execute(sql)
sql = "delete from " + dbname + ".v_increment where Date >= " + date
cursor.execute(sql)
sql = "delete from " + dbname + ".b_increment where Date >= " + date
cursor.execute(sql)
sql = "delete from " + dbname + ".s_increment where Date >= " + date
cursor.execute(sql)

cursor.close()

In [49]:
company_order = pd.read_excel('../期貨公司名稱/大商所/m_company_name.xlsx',encoding='gbk',header=None,index_col=None)
databaseName = 'm_oi_call_option'
# databaseName = 'm_oi_put_option'

cursor = db.cursor()

DATA_DIR = '../各交易所期貨持倉量/大商所/m_call'

for file in os.listdir(DATA_DIR):
    
    date = file.split('_')[0]
    date = datetime.strptime(date,'%Y%m%d')
    
    contract_id = file.split('_')[1].split('.csv')[0]
    
    if (date >= datetime(2018,5,3)):
        split_date = date.strftime('%Y-%m-%d').split('-')
        table_date = split_date[0]+split_date[1]+split_date[2]
   
        data = pd.read_csv(DATA_DIR + '/' + file,encoding='utf-8-sig',engine='python')
        data = data.dropna(how='all')
        
        for i in range(1,len(data)-1):
            for j in range(12):
                if(data.loc[i][j] == "-"):
                    data.loc[i][j] = None  #塞原始資料
                elif( (j==2 or j==3 or j==6 or j==7 or j==10 or j==11) and data.loc[i][j] != None):
                    data.loc[i][j] = data.loc[i][j].replace(',','')
                    
        V_info_insert(data, contract_id, table_date, databaseName)
        B_info_insert(data, contract_id, table_date, databaseName)
        S_info_insert(data, contract_id, table_date, databaseName)
        
        total_volume_insert(data,contract_id,table_date,databaseName)
#         total_insert(data,'m_oi_put_option_origin', file.split('.csv')[0])

In [45]:
table_date

'20180516'

In [15]:
b = company_order.values.tolist()

a = data['Unnamed: 1']
a = pd.DataFrame(list(a))
a = a.values.tolist()

for i in a:
    if i not in b:
        print(i)

['会员简称']
[nan]


In [3]:
def total_insert(data,databaseName,name):       
    cursor = db.cursor()
    
    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + "." + name + "(\
    Rank INT(5),\
    Member CHAR(10),\
    Volume FLOAT(5),\
    Increment FLOAT(5),\
    Rank_1 INT(5),\
    Member1 CHAR(10),\
    Buy_Volume FLOAT(5),\
    Increment_1 FLOAT(5),\
    Rank_2 INT(5),\
    Member2 CHAR(10),\
    Sales_Volume FLOAT(5),\
    Increment_2 FLOAT(5) )"
    
    cursor.execute(sql)
    value=[]

    for i in range(1,len(data)-1):      
        value.append((data.loc[i][0],data.loc[i][1],data.loc[i][2],data.loc[i][3],data.loc[i][4],data.loc[i][5],data.loc[i][6],data.loc[i][7],data.loc[i][8],data.loc[i][9],data.loc[i][10],data.loc[i][11]))

    sql = "INSERT INTO " + databaseName + "." + name +" VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"        
    cursor.executemany(sql,value)
    db.commit()
    cursor.close()

#========總計========
def create_total_volume(databaseName):
    cursor = db.cursor()
    sql = "CREATE TABLE IF NOT EXISTS " + databaseName +".total_volume ( Date DATE, Contract_id VARCHAR(10),\
    V_Valume FLOAT(5),V_Increment FLOAT(5),\
    B_Valume FLOAT(5),B_Increment FLOAT(5),\
    S_Valume FLOAT(5),S_Increment FLOAT(5),primary key(Date,Contract_id))"

    cursor.execute(sql)

def total_volume_insert(data,contract_id,table_date,databaseName):
    
    cursor = db.cursor()
    total_len = len(data)-1
    
    sql = "INSERT INTO " + databaseName + ".total_volume VALUES('"\
    + table_date + "','" + contract_id + "'," \
    + data.loc[total_len][2].replace(',','') +","\
    + data.loc[total_len][3].replace(',','') +","\
    + data.loc[total_len][6].replace(',','') +","\
    + data.loc[total_len][7].replace(',','') +","\
    + data.loc[total_len][10].replace(',','') +","\
    + data.loc[total_len][11].replace(',','') + ")"
    
    cursor.execute(sql)
    db.commit()
#========總計========

#========V/B/S 名次、成交量、增減========
def V_info_create(databaseName, company_order):
    cursor = db.cursor()
    name=""
    for i in range(len(company_order)-1):
        name = name + company_order.loc[i][0] + ' VARCHAR(30),'
    name = name + company_order.loc[len(company_order)-1][0] + " VARCHAR(30)"

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName +".v_rank ( Date DATE, Contract_id VARCHAR(10)," + name + ",primary key(Date,Contract_id)) ENGINE=MyISAM"
    cursor.execute(sql)

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".v_volume ( Date DATE, Contract_id VARCHAR(10)," + name + ",primary key(Date,Contract_id)) ENGINE=MyISAM"
    cursor.execute(sql)

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".v_increment ( Date DATE, Contract_id VARCHAR(10)," + name + ",primary key(Date,Contract_id)) ENGINE=MyISAM"
    cursor.execute(sql)
    cursor.close()

def V_info_insert(data,contract_id,table_date,databaseName):
    cursor = db.cursor()
    
    empty = ""
    empty = empty + table_date + ",'" + contract_id + "'"

    sql = "INSERT INTO " + databaseName + ".v_rank (Date,Contract_id) VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()

    sql = "INSERT INTO " + databaseName + ".v_volume (Date,Contract_id) VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()

    sql = "INSERT INTO " + databaseName + ".v_increment (Date,Contract_id) VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()
    
    for i in range(1,len(data)-1):
        if(data.loc[i][1] == None):
            continue
        
        else:
            sql = "UPDATE " + databaseName + ".v_rank SET " + str(data.loc[i][1]) + "=" + data.loc[i][0] +\
                  " WHERE Date=" + table_date + " and Contract_id = '" + contract_id + "'"
            cursor.execute(sql)
            db.commit()    

    for i in range(1,len(data)-1):
        if(data.loc[i][1] == None):
            continue
            
        else:
            sql = "UPDATE " + databaseName + ".v_volume SET " + str(data.loc[i][1]) + "=" + data.loc[i][2] +\
                  " WHERE Date=" + table_date + " and Contract_id = '" + contract_id + "'"
            cursor.execute(sql)
            db.commit()

    for i in range(1,len(data)-1):
        if(data.loc[i][1] == None):
            continue
            
        else:
            sql = "UPDATE " + databaseName + ".v_increment SET " + data.loc[i][1] + "=" + data.loc[i][3] +\
                  " WHERE Date=" + table_date + " and Contract_id = '" + contract_id + "'"
            cursor.execute(sql)
            db.commit()
    
    cursor.close()
    
def B_info_create(databaseName, company_order):
    cursor = db.cursor()
    name=""
    for i in range(len(company_order)-1):
        name = name + company_order.loc[i][0] + ' TEXT,'
    name = name + company_order.loc[len(company_order)-1][0] + " TEXT"

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".b_rank (Date DATE, Contract_id VARCHAR(10)," + name + ", primary key(Date,Contract_id)) ENGINE=MyISAM"
    cursor.execute(sql)

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".b_oi (Date DATE, Contract_id VARCHAR(10)," + name + ", primary key(Date,Contract_id)) ENGINE=MyISAM"
    cursor.execute(sql)

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".b_increment  (Date DATE, Contract_id VARCHAR(10)," + name + ", primary key(Date,Contract_id)) ENGINE=MyISAM"
    cursor.execute(sql)
    cursor.close()
    
def B_info_insert(data,contract_id,table_date,databaseName):
    cursor = db.cursor()

    empty = ""
    empty = empty + table_date + ",'" + contract_id + "'"
    
    sql = "INSERT INTO " + databaseName + ".b_rank (Date,Contract_id) VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()

    sql = "INSERT INTO " + databaseName + ".b_oi (Date,Contract_id) VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()

    sql = "INSERT INTO " + databaseName + ".b_increment (Date,Contract_id) VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()
    
    for i in range(1,len(data)-1):
        if(data.loc[i][5] == None):
            continue  
        else:
            sql = "UPDATE " + databaseName + ".b_rank SET " + str(data.loc[i][5]) + "=" + data.loc[i][4] +\
                  " WHERE Date=" + table_date + " and Contract_id = '" + contract_id + "'"
            cursor.execute(sql)
            db.commit()    

    for i in range(1,len(data)-1):
        if(data.loc[i][5] == None):
            continue
            
        else:
            sql = "UPDATE " + databaseName + ".b_oi SET " + str(data.loc[i][5]) + "=" + data.loc[i][6] +\
                  " WHERE Date=" + table_date + " and Contract_id = '" + contract_id + "'"
            cursor.execute(sql)
            db.commit()

    for i in range(1,len(data)-1):
        if(data.loc[i][5] == None):
            continue
        
        else:
            sql = "UPDATE " + databaseName + ".b_increment SET " + data.loc[i][5] + "=" + data.loc[i][7] +\
                  " WHERE Date=" + table_date + " and Contract_id = '" + contract_id + "'"
            cursor.execute(sql)
            db.commit()

    cursor.close()
    
def S_info_create(databaseName, company_order):
    cursor = db.cursor()
    name=""
    for i in range(len(company_order)-1):
        name = name + company_order.loc[i][0] + ' TEXT,'
    name = name + company_order.loc[len(company_order)-1][0] + " TEXT"

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName +".s_rank ( Date DATE, Contract_id VARCHAR(10)," + name + ", primary key(Date,Contract_id)) ENGINE=MyISAM"
    cursor.execute(sql)

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".s_oi ( Date DATE, Contract_id VARCHAR(10)," + name + ", primary key(Date,Contract_id)) ENGINE=MyISAM"
    cursor.execute(sql)

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".s_increment ( Date DATE, Contract_id VARCHAR(10)," + name + ", primary key(Date,Contract_id)) ENGINE=MyISAM"
    cursor.execute(sql)
    cursor.close()
    
def S_info_insert(data,contract_id,table_date,databaseName):
    cursor = db.cursor()

    empty = ""
    empty = empty + table_date + ",'" + contract_id + "'"
    
    sql = "INSERT INTO " + databaseName + ".s_rank (Date,Contract_id) VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()

    sql = "INSERT INTO " + databaseName + ".s_oi (Date,Contract_id) VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()

    sql = "INSERT INTO " + databaseName + ".s_increment (Date,Contract_id) VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()
    
    for i in range(1,len(data)-1):
        if(data.loc[i][9] == None):
            continue  
        
        else:
            sql = "UPDATE " + databaseName + ".s_rank SET " + str(data.loc[i][9]) + "=" + data.loc[i][8] +\
                  " WHERE Date=" + table_date + " and Contract_id = '" + contract_id + "'"
            cursor.execute(sql)
            db.commit()    

    for i in range(1,len(data)-1):
        if(data.loc[i][9] == None):
            continue
            
        else:
            sql = "UPDATE " + databaseName + ".s_oi SET " + str(data.loc[i][9]) + "=" + data.loc[i][10] +\
                  " WHERE Date=" + table_date + " and Contract_id = '" + contract_id + "'"
            cursor.execute(sql)
            db.commit()

    for i in range(1,len(data)-1):
        if(data.loc[i][9] == None):
            continue
            
        else:
            sql = "UPDATE " + databaseName +".s_increment SET " + data.loc[i][9] + "=" + data.loc[i][11] +\
                  " WHERE Date=" + table_date + " and Contract_id = '" + contract_id + "'"
            cursor.execute(sql)
            db.commit()

    cursor.close()
    
#========V/B/S 名次、成交量、增減========

# 分割

In [5]:
DATA_DIR = '../各交易所期貨持倉量/大商所/m_option/'

for file in os.listdir(DATA_DIR):
    data = pd.read_csv(DATA_DIR + file,engine='python')
       
    call = data.loc[1:22]
    call.to_csv('../各交易所期貨持倉量/大商所/m_call/' + file, encoding='utf-8-sig',index=None)
    
    put = data.loc[24::]
    put.to_csv('../各交易所期貨持倉量/大商所/m_put/' + file, encoding='utf-8-sig',index=None)